In [1]:
import javalang
import os
import pandas as pd

In [37]:
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score

In [2]:
from collections import Counter

In [3]:
god_class_dictionary = {'class_name': [], 'method_num': []}
with open('resources/xerces2-j-src/org/apache/html/dom/HTMLCollectionImpl.java', 'r') as file:
    tree = javalang.parse.parse(file.read())
    for path, node in tree.filter(javalang.tree.ClassDeclaration):
        method_num = 0
        # print(node.name)
        god_class_dictionary['class_name'].append(node.name)
        for path, node in node.filter(javalang.tree.MethodDeclaration):
            # print(node.name)
            method_num += 1
        god_class_dictionary['method_num'].append(method_num)
            

In [4]:
god_class_dictionary

{'class_name': ['HTMLCollectionImpl', 'CollectionIndex'], 'method_num': [8, 3]}

In [5]:
for path, node in tree.filter(javalang.tree.FieldDeclaration):
    # print(node.declarators)
    for declarator in node.declarators:
        print(declarator.name)

serialVersionUID
ANCHOR
FORM
IMAGE
APPLET
LINK
OPTION
ROW
ELEMENT
AREA
TBODY
CELL
_lookingFor
_topLevel
_index


In [6]:
for path, node in tree.filter(javalang.tree.MethodDeclaration):
    # print(node.declarators) 
    print(node.name)

getLength
item
namedItem
getLength
item
namedItem
recurse
collectionMatch
getIndex
decrement
isZero


In [7]:
for path, node in tree.filter(javalang.tree.MethodDeclaration):
    # print(node.declarators) 
    print('method_name: ', node.name)
    for path, accesses_field in node.filter(javalang.tree.MemberReference):
        print(accesses_field.member)

method_name:  getLength
_topLevel
method_name:  item
index
_topLevel
index
method_name:  namedItem
name
_topLevel
name
method_name:  getLength
topLevel
length
node
node
node
node
length
length
node
node
length
method_name:  item
topLevel
node
node
node
node
node
result
node
index
result
result
node
method_name:  namedItem
topLevel
node
node
node
node
name
node
result
node
name
result
result
node
node
method_name:  recurse
_lookingFor
method_name:  collectionMatch
elem
match
_lookingFor
match
elem
match
elem
match
elem
match
elem
elem
match
elem
match
elem
elem
match
elem
match
elem
match
elem
match
elem
match
elem
match
name
elem
match
match
method_name:  getIndex
_index
method_name:  decrement
_index
method_name:  isZero
_index


In [8]:
for path, node in tree.filter(javalang.tree.MethodDeclaration):
    # print(node.declarators) 
    print('method_name: ', node.name)
    for path, accesses_method in node.filter(javalang.tree.MethodInvocation):
        print(accesses_method.member)

method_name:  getLength
getLength
method_name:  item
item
method_name:  namedItem
namedItem
method_name:  getLength
getFirstChild
collectionMatch
recurse
getLength
getNextSibling
method_name:  item
getFirstChild
collectionMatch
isZero
decrement
recurse
item
getNextSibling
method_name:  namedItem
getFirstChild
collectionMatch
recurse
namedItem
getNextSibling
method_name:  recurse
method_name:  collectionMatch
getAttribute
length
equals
getAttribute
getAttribute
startsWith
getAttribute
length
getTagName
equals
equals
getAttribute
equals
getAttribute
method_name:  getIndex
method_name:  decrement
method_name:  isZero


In [19]:
import javalang

def get_fields(java_class):
    fields = []
    # tree = javalang.parse.parse(java_class)
    for path, node in java_class.filter(javalang.tree.FieldDeclaration):
        for declarator in node.declarators:
            fields.append(declarator.name)
    return fields

def get_methods(java_class):
    methods = []
    methods_name = []
    # tree = javalang.parse.parse(java_class)
    for path, node in java_class.filter(javalang.tree.MethodDeclaration):
        methods.append(node)
        methods_name.append(node.name)
    return methods, methods_name

def get_fields_accessed_by_method(method):
    fields = []
    accessed_field_counter = {}
    # tree = javalang.parse.parse(method)
    for path, node in method.filter(javalang.tree.MemberReference):
        # if isinstance(node.member, javalang.tree.VariableDeclarator):
        if node.qualifier != '':
            fields.append(node.qualifier)
        else:
            fields.append(node.member)
    accessed_field_counter = dict(Counter(fields))
    return fields, accessed_field_counter

def get_methods_accessed_by_method(method):
    methods = []
    accessed_method_counter = {}
    # tree = javalang.parse.parse(method)
    for path, node in method.filter(javalang.tree.MethodInvocation):
        methods.append(node.member)
    accessed_method_counter = dict(Counter(methods))
    return methods, accessed_method_counter


In [20]:
'resources/xerces2-j-src/org/apache/html/dom/HTMLCollectionImpl.java'.split('/')[-1].split('.')[0]

'HTMLCollectionImpl'

In [21]:
nes_dict = {}
# name_f = './resources/xerces2-j-src/org/apache/xerces/dom/CoreDocumentImpl.java'
name_f = './resources/xerces2-j-src/org/apache/xerces/impl/xs/SchemaGrammar.java'
with open(name_f, 'r') as file:
    tree = javalang.parse.parse(file.read())
    for path, class_name in tree.filter(javalang.tree.ClassDeclaration):
        if class_name.name == name_f.split('/')[-1].split('.')[0]:
            method_list, methods_name_list = get_methods(class_name)
            for method in method_list:
                nes_dict[f'{method.name}'] = {}
            field_list = get_fields(class_name)
            for method in method_list:
                accesses_field, accesses_field_dict = get_fields_accessed_by_method(method)
                accesses_method, accesses_method_dict = get_methods_accessed_by_method(method)
                for a_m in accesses_method:
                    if a_m not in methods_name_list:
                        accesses_method_dict.pop(a_m, None)
                nes_dict[f'{method.name}'].update(accesses_method_dict)
                for a_f in accesses_field:
                    if a_f not in field_list:
                        accesses_field_dict.pop(a_f, None)
                nes_dict[f'{method.name}'].update(accesses_field_dict)

In [22]:
methods_name_list

['getGrammarDescription',
 'setImportedGrammars',
 'addGlobalAttributeDecl',
 'addGlobalAttributeDecl',
 'addGlobalAttributeGroupDecl',
 'addGlobalAttributeGroupDecl',
 'addGlobalElementDecl',
 'addGlobalElementDecl',
 'addGlobalElementDeclAll',
 'addGlobalGroupDecl',
 'addGlobalGroupDecl',
 'addGlobalNotationDecl',
 'addGlobalNotationDecl',
 'addGlobalTypeDecl',
 'addGlobalTypeDecl',
 'addGlobalComplexTypeDecl',
 'addGlobalComplexTypeDecl',
 'addGlobalSimpleTypeDecl',
 'addGlobalSimpleTypeDecl',
 'addComplexTypeDecl',
 'addRedefinedGroupDecl',
 'addDocument',
 'getDOMParser',
 'getSAXParser',
 'getGrammarDescription',
 'setImportedGrammars',
 'addGlobalAttributeDecl',
 'addGlobalAttributeDecl',
 'addGlobalAttributeGroupDecl',
 'addGlobalAttributeGroupDecl',
 'addGlobalElementDecl',
 'addGlobalElementDecl',
 'addGlobalElementDeclAll',
 'addGlobalGroupDecl',
 'addGlobalGroupDecl',
 'addGlobalNotationDecl',
 'addGlobalNotationDecl',
 'addGlobalTypeDecl',
 'addGlobalTypeDecl',
 'addGlobal

In [23]:
field_list

['fTargetNamespace',
 'fGlobalAttrDecls',
 'fGlobalAttrGrpDecls',
 'fGlobalElemDecls',
 'fGlobalGroupDecls',
 'fGlobalNotationDecls',
 'fGlobalIDConstraintDecls',
 'fGlobalTypeDecls',
 'fGlobalAttrDeclsExt',
 'fGlobalAttrGrpDeclsExt',
 'fGlobalElemDeclsExt',
 'fGlobalGroupDeclsExt',
 'fGlobalNotationDeclsExt',
 'fGlobalIDConstraintDeclsExt',
 'fGlobalTypeDeclsExt',
 'fAllGlobalElemDecls',
 'fGrammarDescription',
 'fAnnotations',
 'fNumAnnotations',
 'fSymbolTable',
 'fSAXParser',
 'fDOMParser',
 'fIsImmutable',
 'BASICSET_COUNT',
 'FULLSET_COUNT',
 'GRAMMAR_XS',
 'GRAMMAR_XSI',
 'EXTENDED_SCHEMA_FACTORY_CLASS',
 'INSTANCE',
 'fImported',
 'INITIAL_SIZE',
 'INC_SIZE',
 'fCTCount',
 'fComplexTypeDecls',
 'fCTLocators',
 'REDEFINED_GROUP_INIT_SIZE',
 'fRGCount',
 'fRedefinedGroupDecls',
 'fRGLocators',
 'fFullChecked',
 'fSubGroupCount',
 'fSubGroups',
 'fAnyType',
 'SG_SchemaNS',
 'SG_SchemaNSExtended',
 'fAnySimpleType',
 'SG_XSI',
 'MAX_COMP_IDX',
 'GLOBAL_COMP',
 'fComponents',
 'fCom

In [24]:
nes_dict

{'getGrammarDescription': {'fGrammarDescription': 1},
 'setImportedGrammars': {'fImported': 1},
 'addGlobalAttributeDecl': {'getNamespaceItem': 1},
 'addGlobalAttributeGroupDecl': {'getNamespaceItem': 1},
 'addGlobalElementDecl': {'getNamespaceItem': 1},
 'addGlobalElementDeclAll': {'resize': 1,
  'fSubGroupCount': 3,
  'fSubGroups': 4,
  'INC_SIZE': 1},
 'addGlobalGroupDecl': {'getNamespaceItem': 1},
 'addGlobalNotationDecl': {'getNamespaceItem': 1},
 'addGlobalTypeDecl': {'getNamespaceItem': 1},
 'addGlobalComplexTypeDecl': {'getNamespaceItem': 1},
 'addGlobalSimpleTypeDecl': {'getNamespaceItem': 1},
 'addComplexTypeDecl': {'resize': 2,
  'fCTCount': 5,
  'fComplexTypeDecls': 4,
  'INC_SIZE': 2,
  'fCTLocators': 3},
 'addRedefinedGroupDecl': {'resize': 2,
  'fRGCount': 6,
  'fRedefinedGroupDecls': 5,
  'fRGLocators': 3},
 'addDocument': {'fDocuments': 2, 'fLocations': 1},
 'getDOMParser': {'fDOMParser': 2, 'fSymbolTable': 1},
 'getSAXParser': {'fSAXParser': 2, 'fSymbolTable': 1},
 'c

In [25]:
row_list = list(set(methods_name_list))
column_list = list(set(methods_name_list)) + list(set(field_list))
df = pd.DataFrame(columns=column_list, index=row_list)

In [26]:
for method_name in df.index:
    for field_name in df.columns:
        if method_name in nes_dict and field_name in nes_dict[method_name]:
            df.loc[method_name, field_name] = 1
        else:
            df.loc[method_name, field_name] = 0


In [27]:
nes_dict

{'getGrammarDescription': {'fGrammarDescription': 1},
 'setImportedGrammars': {'fImported': 1},
 'addGlobalAttributeDecl': {'getNamespaceItem': 1},
 'addGlobalAttributeGroupDecl': {'getNamespaceItem': 1},
 'addGlobalElementDecl': {'getNamespaceItem': 1},
 'addGlobalElementDeclAll': {'resize': 1,
  'fSubGroupCount': 3,
  'fSubGroups': 4,
  'INC_SIZE': 1},
 'addGlobalGroupDecl': {'getNamespaceItem': 1},
 'addGlobalNotationDecl': {'getNamespaceItem': 1},
 'addGlobalTypeDecl': {'getNamespaceItem': 1},
 'addGlobalComplexTypeDecl': {'getNamespaceItem': 1},
 'addGlobalSimpleTypeDecl': {'getNamespaceItem': 1},
 'addComplexTypeDecl': {'resize': 2,
  'fCTCount': 5,
  'fComplexTypeDecls': 4,
  'INC_SIZE': 2,
  'fCTLocators': 3},
 'addRedefinedGroupDecl': {'resize': 2,
  'fRGCount': 6,
  'fRedefinedGroupDecls': 5,
  'fRGLocators': 3},
 'addDocument': {'fDocuments': 2, 'fLocations': 1},
 'getDOMParser': {'fDOMParser': 2, 'fSymbolTable': 1},
 'getSAXParser': {'fSAXParser': 2, 'fSymbolTable': 1},
 'c

In [28]:
df

,createAnnotationElementDecl,getTypeDefinition,getUncheckedCTLocators,getAnnotations,getGlobalElementDecl,getGlobalGroupDecl,resize,addDocument,getComponentsExt,createAttrGrp,...,fAnySimpleType,fComponents,fAllGlobalElemDecls,FULLSET_COUNT,fGlobalNotationDeclsExt,fGlobalIDConstraintDecls,fGlobalGroupDeclsExt,fGlobalElemDeclsExt,fSubGroupCount,fDOMParser
createAnnotationElementDecl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
getTypeDefinition,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
getUncheckedCTLocators,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
getAnnotations,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
getGlobalElementDecl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
getImportedGrammars,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
getIDConstraintDecl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
addIDConstraintDecl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
getRedefinedGroupDecls,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
df.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

In [41]:
file_name = input('Enter the name of the file containing the feature vectors: ')

In [173]:
file_name

'SchemaGrammar.csv'

In [159]:
df = pd.read_csv(file_name, index_col=0)

In [160]:
df

,getGlobalAttributeDecl,setImportedGrammars,setValues,getGrammarDescription,addGlobalNotationDecl,addRedefinedGroupDecl,getSAXParser,getSubstitutionGroups,setContainsTypeID,getNotationDeclaration,...,fGlobalAttrGrpDeclsExt,fGlobalTypeDecls,fLocations,fSAXParser,fRGLocators,fGlobalAttrDeclsExt,fTargetNamespace,INC_SIZE,fCTLocators,fIsImmutable
getGlobalAttributeDecl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
setImportedGrammars,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
setValues,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
getGrammarDescription,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
addGlobalNotationDecl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
addIDConstraintDecl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
getDOMParser,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
addGlobalComplexTypeDecl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
setUncheckedTypeNum,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [161]:
df.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [162]:
def k_means_clustering(feature_vectors, k):
    kmeans = KMeans(n_clusters=k, random_state=0).fit(feature_vectors)
    return kmeans.labels_


In [163]:
labels = k_means_clustering(df.values, 3)

In [164]:
labels

array([0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int32)

In [165]:
df.index

Index(['getGlobalAttributeDecl', 'setImportedGrammars', 'setValues',
       'getGrammarDescription', 'addGlobalNotationDecl',
       'addRedefinedGroupDecl', 'getSAXParser', 'getSubstitutionGroups',
       'setContainsTypeID', 'getNotationDeclaration', 'getGlobalNotationDecl',
       'hasIDConstraints', 'createUnboundedAnyWildcardSequenceParticle',
       'getDocumentLocations', 'getRedefinedGroupDecls',
       'createChoiceElementParticle', 'toXSModel', 'addGlobalTypeDecl',
       'createAnyLaxWildcardParticle', 'getGlobalElementDecl', 'reset',
       'addGlobalAttributeDecl', 'createUnboundedModelGroupParticle',
       'addGlobalGroupDecl', 'getAttributeDeclaration',
       'getUncheckedCTLocators', 'getAttributeGroup', 'addComplexTypeDecl',
       'getS4SGrammar', 'getGlobalGroupDecl', 'createAnnotationElementDecl',
       'isNamespaceAware', 'getGlobalAttributeGroupDecl', 'setIsAnonymous',
       'resize', 'getNamespaceItem', 'addGlobalSimpleTypeDecl',
       'createParticle', 'set

In [166]:
data = {'Method_Name': df.index, 'labels': labels}
df_lables = pd.DataFrame(data)

In [171]:
df_lables = pd.read_csv('SchemaGrammar_clustering_hierarchical.csv')

In [167]:
df_lables

,Method_Name,labels
0,getGlobalAttributeDecl,0
1,setImportedGrammars,0
2,setValues,0
3,getGrammarDescription,0
4,addGlobalNotationDecl,0
...,...,...
64,addIDConstraintDecl,0
65,getDOMParser,0
66,addGlobalComplexTypeDecl,0
67,setUncheckedTypeNum,0


In [168]:
df_lables.labels

0     0
1     0
2     0
3     0
4     0
     ..
64    0
65    0
66    0
67    0
68    0
Name: labels, Length: 69, dtype: int32

In [169]:
df.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [170]:
silhouette_score(df.values, df_lables.labels, metric='euclidean')

0.3693855635609086

In [174]:
df = pd.read_csv(file_name, index_col=0)

In [176]:
df.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [177]:
def k_means_score(feature_vectors, k):
    kmeans = KMeans(n_clusters=k, random_state=0).fit(feature_vectors)
    score = silhouette_score(df.values, kmeans.labels_, metric='euclidean')
    return score

In [194]:
k_scores = []
k = []
for i in range(2, 31):
    score = k_means_score(df.values, i)
    k_scores.append(score)
    k.append(i)
data = {'num_clusters': k, 'silhouette_score': k_scores}
df_k_scores = pd.DataFrame(data)

In [195]:
df_k_scores

,num_clusters,silhouette_score
0,2,0.558978
1,3,0.369386
2,4,0.438617
3,5,0.450783
4,6,0.512679
5,7,0.443591
6,8,0.428835
7,9,0.446818
8,10,0.480022
9,11,0.498901


In [196]:
k_scores

[0.5589775836760922,
 0.3693855635609086,
 0.43861715891158376,
 0.45078306859631867,
 0.5126790725359008,
 0.44359121780803684,
 0.42883521908353206,
 0.4468178213459185,
 0.48002231844781945,
 0.49890131089533934,
 0.4957547840892742,
 0.5041756135590847,
 0.550058821780288,
 0.5474552633099886,
 0.5511933296102842,
 0.5693564148479823,
 0.5797905273308834,
 0.5845325129773261,
 0.6229775871339951,
 0.6260082094983791,
 0.6321702838953219,
 0.6387857238897087,
 0.624930854249718,
 0.6321322649882681,
 0.6312943369931585,
 0.6392447803621685,
 0.6383823954192047,
 0.6470248116337068,
 0.6564187423016442]

In [178]:
for i in range(2, 60):
    print(f'k = {i}: {k_means_score(df.values, i)}')

k = 2: 0.5589775836760922
k = 3: 0.3693855635609086
k = 4: 0.43861715891158376
k = 5: 0.45078306859631867
k = 6: 0.5126790725359008
k = 7: 0.44359121780803684
k = 8: 0.42883521908353206
k = 9: 0.4468178213459185
k = 10: 0.48002231844781945
k = 11: 0.49890131089533934
k = 12: 0.4957547840892742
k = 13: 0.5041756135590847
k = 14: 0.550058821780288
k = 15: 0.5474552633099886
k = 16: 0.5511933296102842
k = 17: 0.5693564148479823
k = 18: 0.5797905273308834
k = 19: 0.5845325129773261
k = 20: 0.6229775871339951
k = 21: 0.6260082094983791
k = 22: 0.6321702838953219
k = 23: 0.6387857238897087
k = 24: 0.624930854249718
k = 25: 0.6321322649882681
k = 26: 0.6312943369931585
k = 27: 0.6392447803621685
k = 28: 0.6383823954192047
k = 29: 0.6470248116337068
k = 30: 0.6564187423016442
k = 31: 0.6666666666666666
k = 32: 0.6666666666666666
k = 33: 0.6666666666666666
k = 34: 0.6666666666666666
k = 35: 0.6666666666666666
k = 36: 0.6666666666666666
k = 37: 0.6666666666666666
k = 38: 0.6666666666666666
k = 3

/Users/navdeepsinghbedi/project-01-god-classes-navdeeps350/god_class_env/lib/python3.11/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (31) found smaller than n_clusters (32). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/Users/navdeepsinghbedi/project-01-god-classes-navdeeps350/god_class_env/lib/python3.11/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (31) found smaller than n_clusters (33). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/Users/navdeepsinghbedi/project-01-god-classes-navdeeps350/god_class_env/lib/python3.11/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (31) found smaller than n_clusters (34). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/Users/navdeepsinghbedi/project-01-god-classes-navdeeps350/god_class_env/lib/python3.11/site-package

In [139]:
n_clusters = 3
clusterer = KMeans(n_clusters=n_clusters, random_state=10)
cluster_labels = clusterer.fit_predict(df.values)


In [140]:
cluster_labels

array([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1], dtype=int32)

In [155]:
kmeans = KMeans(n_clusters = n_clusters, random_state = 0)
kmeans.fit(df.values)

KMeans(n_clusters=3, random_state=0)

In [156]:
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int32)

In [157]:
silhouette_score(df.values, kmeans.labels_, metric='euclidean')

0.3693855635609086

In [117]:
agglomerative_clustering= AgglomerativeClustering(n_clusters=3)
agglomerative_clustering.fit(df.values)

AgglomerativeClustering(n_clusters=3)

In [118]:
agglomerative_clustering.labels_

array([2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 2])

In [119]:
silhouette_score(df.values, agglomerative_clustering.labels_, metric='euclidean')

0.45398116664845933

In [109]:
god_class_dictionary = {'class_name': [], 'path': [], 'method_num': []}
for (dirpath, dirnames, filenames) in os.walk("./resources/xerces2-j-src"):
    for filename in filenames:
        if filename.endswith('.java'):
            # print(dirpath, filename)
            with open(os.path.join(dirpath, filename), 'r') as file:
                tree = javalang.parse.parse(file.read())
                for path, node in tree.filter(javalang.tree.ClassDeclaration):
                    method_num = 0
                    # print(node.name)
                    god_class_dictionary['class_name'].append(node.name)
                    god_class_dictionary['path'].append(os.path.join(dirpath, filename))
                    for path, method in node.filter(javalang.tree.MethodDeclaration):
                        # print(node.name)
                        method_num += 1
                    god_class_dictionary['method_num'].append(method_num)      
                          

In [110]:
god_class_dictionary

{'class_name': ['HTMLAreaElementImpl',
  'HTMLLegendElementImpl',
  'HTMLFieldSetElementImpl',
  'HTMLModElementImpl',
  'HTMLMapElementImpl',
  'HTMLHRElementImpl',
  'HTMLStyleElementImpl',
  'HTMLHeadingElementImpl',
  'HTMLOptionElementImpl',
  'HTMLImageElementImpl',
  'HTMLFrameElementImpl',
  'HTMLBRElementImpl',
  'HTMLIFrameElementImpl',
  'HTMLLabelElementImpl',
  'HTMLTitleElementImpl',
  'HTMLTableRowElementImpl',
  'HTMLOptGroupElementImpl',
  'HTMLTableSectionElementImpl',
  'HTMLLIElementImpl',
  'HTMLDOMImplementationImpl',
  'HTMLDocumentImpl',
  'HTMLSelectElementImpl',
  'HTMLMenuElementImpl',
  'HTMLMetaElementImpl',
  'HTMLBuilder',
  'HTMLDivElementImpl',
  'HTMLAnchorElementImpl',
  'HTMLTextAreaElementImpl',
  'HTMLBaseFontElementImpl',
  'ObjectFactory',
  'ConfigurationError',
  'HTMLAppletElementImpl',
  'SecuritySupport',
  'HTMLTableCaptionElementImpl',
  'HTMLObjectElementImpl',
  'NameNodeListImpl',
  'HTMLPreElementImpl',
  'HTMLCollectionImpl',
  'Colle

In [111]:
god_class_dataframe = pd.DataFrame.from_dict(god_class_dictionary)

In [112]:
god_class_dataframe

,class_name,path,method_num
0,HTMLAreaElementImpl,./resources/xerces2-j-src/org/apache/html/dom/...,16
1,HTMLLegendElementImpl,./resources/xerces2-j-src/org/apache/html/dom/...,4
2,HTMLFieldSetElementImpl,./resources/xerces2-j-src/org/apache/html/dom/...,0
3,HTMLModElementImpl,./resources/xerces2-j-src/org/apache/html/dom/...,4
4,HTMLMapElementImpl,./resources/xerces2-j-src/org/apache/html/dom/...,4
...,...,...,...
722,WMLAnchorElementImpl,./resources/xerces2-j-src/org/apache/wml/dom/W...,8
723,WMLTimerElementImpl,./resources/xerces2-j-src/org/apache/wml/dom/W...,8
724,WMLHeadElementImpl,./resources/xerces2-j-src/org/apache/wml/dom/W...,4
725,WMLBElementImpl,./resources/xerces2-j-src/org/apache/wml/dom/W...,6


In [113]:
mean = god_class_dataframe['method_num'].mean()

In [114]:
standard_dev = god_class_dataframe['method_num'].std()

In [115]:
mean + 6 * standard_dev

118.26335085092768

In [116]:
god_class_dataframe.loc[god_class_dataframe['method_num'] > mean + 6 * standard_dev, ['class_name', 'path', 'method_num']].to_csv('god_classes.csv', index=False)

In [117]:
god_classes = pd.read_csv('god_classes.csv')

In [118]:
god_classes

,class_name,path,method_num
0,SchemaGrammar,./resources/xerces2-j-src/org/apache/xerces/im...,141
1,XSDHandler,./resources/xerces2-j-src/org/apache/xerces/im...,129
2,XSSimpleTypeDecl,./resources/xerces2-j-src/org/apache/xerces/im...,149
3,XIncludeHandler,./resources/xerces2-j-src/org/apache/xerces/xi...,122
4,CoreDocumentImpl,./resources/xerces2-j-src/org/apache/xerces/do...,125


In [119]:
god_classes['path'].tolist()

['./resources/xerces2-j-src/org/apache/xerces/impl/xs/SchemaGrammar.java',
 './resources/xerces2-j-src/org/apache/xerces/impl/xs/traversers/XSDHandler.java',
 './resources/xerces2-j-src/org/apache/xerces/impl/dv/xs/XSSimpleTypeDecl.java',
 './resources/xerces2-j-src/org/apache/xerces/xinclude/XIncludeHandler.java',
 './resources/xerces2-j-src/org/apache/xerces/dom/CoreDocumentImpl.java']

In [121]:
god_class_dataframe.iloc[406]

class_name                                      XIncludeHandler
path          ./resources/xerces2-j-src/org/apache/xerces/xi...
method_num                                                  122
Name: 406, dtype: object

In [80]:
columns = set()
rows = []
for (dirpath, dirnames, filenames) in os.walk("./resources/xerces2-j-src"):
    for filename in filenames:
        if filename.endswith('.java'):
            # print(filename.split('.')[0])
            with open(os.path.join(dirpath, filename), 'r') as file:
                tree = javalang.parse.parse(file.read())
                for path, node in tree.filter(javalang.tree.ClassDeclaration):
                    if node.name == filename.split('.')[0]:
                        for path, method in node.filter(javalang.tree.MethodDeclaration):
                            rows.append(method.name)
                            for path, accesses_method in method.filter(javalang.tree.MethodInvocation):
                                columns.add(accesses_method.member)
                            for path, accesses_field in method.filter(javalang.tree.MemberReference):
                                columns.add(accesses_field.member)

In [81]:
rows

['getAccessKey',
 'setAccessKey',
 'getAlt',
 'setAlt',
 'getCoords',
 'setCoords',
 'getHref',
 'setHref',
 'getNoHref',
 'setNoHref',
 'getShape',
 'setShape',
 'getTabIndex',
 'setTabIndex',
 'getTarget',
 'setTarget',
 'getAccessKey',
 'setAccessKey',
 'getAlign',
 'setAlign',
 'getCite',
 'setCite',
 'getDateTime',
 'setDateTime',
 'getAreas',
 'getName',
 'setName',
 'cloneNode',
 'getAlign',
 'setAlign',
 'getNoShade',
 'setNoShade',
 'getSize',
 'setSize',
 'getWidth',
 'setWidth',
 'getDisabled',
 'setDisabled',
 'getMedia',
 'setMedia',
 'getType',
 'setType',
 'getAlign',
 'setAlign',
 'getDefaultSelected',
 'setDefaultSelected',
 'getText',
 'setText',
 'getIndex',
 'setIndex',
 'getDisabled',
 'setDisabled',
 'getLabel',
 'setLabel',
 'getSelected',
 'setSelected',
 'getValue',
 'setValue',
 'getLowSrc',
 'setLowSrc',
 'getSrc',
 'setSrc',
 'getName',
 'setName',
 'getAlign',
 'setAlign',
 'getAlt',
 'setAlt',
 'getBorder',
 'setBorder',
 'getHeight',
 'setHeight',
 'getHs

In [82]:
columns

{'getXsiNil',
 'getTableViewBucket',
 'startOffset',
 'fFIXEDSymbol',
 'combinedFeatureIds',
 'entityDepth',
 'svalue',
 'ROW',
 'UNSIGNEDSHORT_DT',
 'uniqueOrKey',
 'fExtraDVs',
 'EXTERNAL_GENERAL_ENTITIES',
 'newsize',
 'maxNodeLimit',
 'normalizeDefaultAttrValue',
 'exists',
 'addGrammarComponents',
 'signalSplitData',
 'fAttributeChecker',
 '_elementStates',
 'SCANNER_STATE_DTD_EXTERNAL',
 'host',
 'START_PUNCTUATION',
 'getAttrDeclaration',
 'evtName',
 'CONTROL',
 'ELT_MININCLUSIVE',
 'getAnnotations',
 'SIZE',
 'getParentNode',
 'STATE_CHILD',
 'createStartElement',
 'typeList1Length',
 'newdoctype',
 'setImportedGrammars',
 'DV_LIST',
 'fillProcessingInstruction',
 'isCompactedForUPA',
 'shiftTable',
 'fSchemaElement',
 'negative',
 'dvTemp',
 'fFinalSet',
 'endDTD',
 'ELEMENT_NODE',
 'fValidator',
 'subbegin',
 'fSAXLocatorWrapper',
 'getYears',
 'fElementChildrenOffsetStack',
 'deep',
 'tnsAtt',
 'ident',
 'XINCLUDE_ATTR_HREF',
 'formAtt',
 'fDeferredEntityDecl',
 'saveUnnorm

In [68]:
columns = []
rows = []
columns_accesses_field = []
columns_accesses_method = []
nested_dictionary = {}

for (dirpath, dirnames, filenames) in os.walk("./resources/xerces2-j-src"):
    for filename in filenames:
        if filename.endswith('.java'):
            # print(filename.split('.')[0])
            with open(os.path.join(dirpath, filename), 'r') as file:
                tree = javalang.parse.parse(file.read())
                for path, class_ in tree.filter(javalang.tree.ClassDeclaration):
                    if class_.name == filename.split('.')[0]:
                        method_list, methods_name_list = get_methods(class_)
                        rows.extend(methods_name_list)
                        # field_list = get_fields(class_)
                        for method in method_list:
                            accesses_field,  accesses_field_dict= get_fields_accessed_by_method(method)
                            accesses_method, accesses_method_dict = get_methods_accessed_by_method(method)
                            nested_dictionary[f'{method.name}'] = {}
                            nested_dictionary[f'{method.name}'].update(accesses_field_dict)
                            nested_dictionary[f'{method.name}'].update(accesses_method_dict)
                            columns_accesses_field.extend(accesses_field)
                            columns_accesses_method.extend(accesses_method)
columns = columns_accesses_field + columns_accesses_method

                            
                        # for path, method in node.filter(javalang.tree.MethodDeclaration):
                        #     rows.append(method.name)
                        #     for path, accesses_method in method.filter(javalang.tree.MethodInvocation):
                        #         columns.add(accesses_method.member)
                        #     for path, accesses_field in method.filter(javalang.tree.MemberReference):
                        #         columns.add(accesses_field.member)

In [69]:
nested_dictionary

{'getAccessKey': {'accessKey': 4,
  'getAttribute': 5,
  'length': 2,
  'substring': 2,
  'setAttribute': 5,
  'getBinary': 1,
  'parseInt': 1,
  'valueOf': 1,
  'capitalize': 1},
 'setAccessKey': {'accessKey': 3,
  'getAttribute': 5,
  'length': 2,
  'substring': 2,
  'setAttribute': 5,
  'getBinary': 1,
  'parseInt': 1,
  'valueOf': 1,
  'capitalize': 1},
 'getAlt': {'setAttribute': 11, 'getAttribute': 11},
 'setAlt': {'newValue': 1, 'setAttribute': 11, 'getAttribute': 11},
 'getCoords': {'getAttribute': 12,
  'length': 2,
  'substring': 2,
  'setAttribute': 12,
  'capitalize': 1,
  'getInteger': 1,
  'valueOf': 1},
 'setCoords': {'coords': 1,
  'getAttribute': 12,
  'length': 2,
  'substring': 2,
  'setAttribute': 12,
  'capitalize': 1,
  'getInteger': 1,
  'valueOf': 1},
 'getHref': {'setAttribute': 6, 'getAttribute': 6},
 'setHref': {'newValue': 1, 'setAttribute': 6, 'getAttribute': 6},
 'getNoHref': {'getAttribute': 7,
  'length': 2,
  'substring': 2,
  'setAttribute': 8,
  'getB

In [66]:
len(rows)

7690

In [42]:
columns

['accessKey',
 'accessKey',
 'accessKey',
 'accessKey',
 'accessKey',
 'accessKey',
 'accessKey',
 'alt',
 'coords',
 'href',
 'noHref',
 'shape',
 'tabIndex',
 'target',
 'accessKey',
 'accessKey',
 'accessKey',
 'accessKey',
 'accessKey',
 'accessKey',
 'accessKey',
 'align',
 'cite',
 'dateTime',
 '_areas',
 '_areas',
 'AREA',
 '_areas',
 'name',
 'deep',
 '_areas',
 'clonedNode',
 'align',
 'noShade',
 'size',
 'width',
 'disabled',
 'media',
 'type',
 'align',
 'defaultSelected',
 'child',
 'child',
 'child',
 'child',
 'child',
 'child',
 'child',
 'next',
 'child',
 'child',
 'next',
 'text',
 'parent',
 'parent',
 'parent',
 'parent',
 'parent',
 'options',
 'parent',
 'i',
 'i',
 'i',
 'i',
 'i',
 'parent',
 'parent',
 'parent',
 'parent',
 'parent',
 'options',
 'parent',
 'index',
 'item',
 'index',
 'item',
 'disabled',
 'label',
 'selected',
 'value',
 'lowSrc',
 'src',
 'name',
 'align',
 'alt',
 'border',
 'height',
 'hspace',
 'isMap',
 'longDesc',
 'useMap',
 'vspace',